In [1]:
from mistralai import Mistral
import os
import pandas as pd
from pathlib import Path
from sqlutils import sqlutils

In [2]:
class MistralAPI:
    """
    A client for interacting with the MistralAI API.

    Attributes:
        client (Mistral): The Mistral client instance.
        model (str): The model to use for queries.
    """

    def __init__(self, model: str) -> None:
        """
        Initializes the MistralAPI with the given model.

        Args:
            model (str): The model to use for queries.

        Raises:
            ValueError: If the MISTRAL_API_KEY environment variable is not set.
        """
        api_key = os.getenv("MISTRAL_API_KEY")
        if not api_key:
            raise ValueError(
                "No MISTRAL_API_KEY as environment variable, please set it!"
            )
        self.client = Mistral(api_key=api_key)
        self.model = model

    def query(self, query: str, temperature: float = 0.5) -> str:
        """
        Sends a query to the MistralAI API and returns the response.

        Args:
            query (str): The input query to send to the model.
            temperature (float, optional): The temperature parameter for controlling
                                          the randomness of the output. Defaults to 0.5.

        Returns:
            str: The response from the API.
        """
        chat_response = self.client.chat.complete(
            model=self.model,
            temperature=temperature,
            messages=[
                {
                    "role": "user",
                    "content": query,
                },
            ],
        )
        return chat_response.choices[0].message.content

In [3]:
# Déterminer le chemin du script
script_path = Path().resolve()

# # Select des avis depuis la base de données et depuis le chemin du script
bdd = sqlutils(script_path / "../../data/friands.db")
success, t_avis = bdd.select("SELECT * FROM avis")

if not success:
    print("Erreur lors de l'extraction des avis depuis la base de données")
    print(t_avis)

# # Insérer les champs extraits de la base de données dans un dataframe
df = pd.DataFrame(
    t_avis,
    columns=[
        "id_avis",
        "id_restaurant",
        "nom_utilisateur",
        "note_restaurant",
        "date_avis",
        "titre_avis",
        "contenu_avis",
        "label",
    ],
)

In [4]:
# Sélectionner les id_restaurant et les noms des restaurants depuis la table restaurant
success, t_resto = bdd.select("SELECT id_restaurant, nom FROM restaurants")

if not success:
    print(
        "Erreur lors de l'extraction des informations des restaurants depuis la base de données"
    )
    print(t_resto)
else:
    # Ajouter les infos au dataframe df
    df_resto = pd.DataFrame(t_resto, columns=["id_restaurant", "nom_restaurant"])
    df = pd.merge(df, df_resto, on="id_restaurant")

In [5]:
# joindre tous les avis pour un restaurant donné
df_grouped = (
    df.groupby("id_restaurant")
    .agg(
        {
            "nom_restaurant": "first",
            "contenu_avis": lambda x: " ".join(x),
        }
    )
    .reset_index()
)

In [6]:
os.environ["MISTRAL_API_KEY"] = ""

In [7]:
model_mistral = MistralAPI(model="ministral-8b-latest")


def split_text(text, max_length):
    """
    Splits the text into chunks of maximum length.
    """
    words = text.split()
    for i in range(0, len(words), max_length):
        yield " ".join(words[i : i + max_length])


for index, row in df_grouped.iterrows():
    reviews = row["contenu_avis"]
    chunks = list(split_text(reviews, 131000))  # Adjust the chunk size as needed
    summaries = []
    for chunk in chunks:
        summary = model_mistral.query(f"Résume les avis suivants : '{chunk}'")
        summaries.append(summary)
    full_summary = " ".join(summaries)
    print(f"Restaurant: {row['nom_restaurant']}")
    print(f"Résumé: {full_summary}")
    print("\n")

Restaurant: KUMA cantine
Résumé: ### Résumé des avis

#### Avis Positifs
- **Ambiance et Décoration**: La plupart des avis soulignent la décoration typique et l'ambiance chaleureuse du restaurant.
- **Cuisine**: Les plats sont généralement décrits comme excellents, savoureux et authentiques. Les gyoza, ramen, takoyaki, et tiramisu au thé vert sont particulièrement appréciés.
- **Service**: Le service est souvent décrit comme rapide, agréable et sympathique.
- **Rapport Qualité-Prix**: La majorité des avis trouvent le rapport qualité-prix très bon.
- **Découverte Culinaire**: Beaucoup d'avis mettent en avant la découverte de nouvelles saveurs et la qualité des produits utilisés.

#### Avis Négatifs
- **Problèmes de Cuisine**: Certains avis mentionnent des plats décevants, notamment le tonkatsu et les crevettes panées, qui ne sont pas à la hauteur des attentes.
- **Problèmes de Service**: Quelques avis signalent des problèmes de service, comme des temps d'attente longs ou des erreurs de 

KeyboardInterrupt: 

In [6]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(
    ministral8b.query(
        f"Résume les avis suivants en ne proposant qu'une conclusion succinte : '{review1}'"
    )
)

Kuma Izakaya est un restaurant japonais authentique et savoureux, offrant une cuisine traditionnelle et des plats variés. Les avis sont globalement positifs, soulignant la qualité des plats, le service chaleureux et l'accueil sympathique. Les desserts, notamment le tiramisu au thé vert, sont particulièrement appréciés. La décoration typique japonaise et l'ambiance cosy contribuent à l'expérience culinaire. Les prix sont raisonnables compte tenu de la qualité des plats. Cependant, certains avis mentionnent des problèmes de service ou de qualité des plats, mais dans l'ensemble, Kuma Izakaya est recommandé pour une expérience culinaire japonaise authentique.


In [7]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(ministral8b.query(f"Résume les avis suivants : '{review1}'", temperature=0))

### Résumé des avis

#### Avis Positifs
- **Ambiance et Décoration**: La plupart des avis soulignent l'ambiance chaleureuse et la décoration typiquement japonaise du restaurant.
- **Cuisine**: Les plats sont généralement décrits comme excellents, savoureux et authentiques. Les ramens, gyozas, takoyakis, et autres spécialités japonaises sont particulièrement appréciés.
- **Service**: Le service est souvent décrit comme rapide, agréable et efficace. Les serveurs sont sympathiques et accueillants.
- **Rapport Qualité-Prix**: La majorité des avis trouvent que le rapport qualité-prix est excellent, avec des prix raisonnables pour la qualité des plats.
- **Desserts**: Les desserts, notamment le tiramisu au thé vert, sont souvent mentionnés comme étant délicieux.
- **Accueil**: L'accueil est généralement chaleureux et sympathique, avec un personnel attentionné et serviable.

#### Avis Négatifs
- **Problèmes de Cuisine**: Certains avis mentionnent des problèmes de qualité de la cuisine, comme 

In [8]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(ministral8b.query(f"Résume les avis suivants : '{review1}'", temperature=0))

### Résumé des avis

#### Avis Positifs
- **Ambiance et Décoration**: La plupart des avis soulignent l'ambiance chaleureuse et la décoration typiquement japonaise du restaurant.
- **Cuisine**: Les plats sont généralement décrits comme excellents, savoureux et authentiques. Les ramens, gyozas, takoyakis, et autres spécialités japonaises sont particulièrement appréciés.
- **Service**: Le service est souvent décrit comme rapide, agréable et efficace. Les serveurs sont sympathiques et accueillants.
- **Rapport Qualité-Prix**: La majorité des avis trouvent que le rapport qualité-prix est excellent, avec des prix raisonnables pour la qualité des plats.
- **Desserts**: Les desserts, notamment le tiramisu au thé vert, sont souvent mentionnés comme étant délicieux.
- **Accueil**: L'accueil est généralement chaleureux et sympathique, avec un personnel attentionné et serviable.

#### Avis Négatifs
- **Problèmes de Cuisine**: Certains avis mentionnent des problèmes de qualité de la cuisine, comme 

In [10]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(
    ministral8b.query(
        f"Produire une liste de 5 mots clés à partir des avis suivants : '{review1}'",
        temperature=0.2,
    )
)

## Résultat non concluant

Voici une liste de 5 mots clés à partir des avis suivants :

1. **Authentique** : Les avis mettent en avant l'authenticité de la cuisine japonaise proposée par le restaurant.
2. **Décor** : La décoration typique et épurée du restaurant est souvent mentionnée comme un point positif.
3. **Service** : Le service est décrit comme rapide, agréable et efficace dans la plupart des avis.
4. **Plats** : Les plats sont souvent qualifiés de délicieux, savoureux et bien présentés.
5. **Rapport qualité-prix** : Le rapport qualité-prix est souvent souligné comme étant très bon, avec des prix raisonnables pour la qualité des plats.
